<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/Convolutions/Breast_cancer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Breast cancer segmentation

## Installs & Imports

In [1]:
%%capture
#pip install catalyst

In [2]:
#import albumentations as albs
import cv2
import io
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

from glob import glob
from sklearn.model_selection import train_test_split
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from IPython import display

#from catalyst import dl, utils

## Download dataset

In [3]:
!curl -JLO 'https://www.dropbox.com/scl/fi/gs3kzp6b8k6faf667m5tt/breast-cancer-cells-segmentation.zip?rlkey=md3mzikpwrvnaluxnhms7r4zn'
!unzip breast-cancer-cells-segmentation.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256    0   256    0     0    300      0 --:--:-- --:--:-- --:--:--   300
100   491    0   491    0     0    254      0 --:--:--  0:00:01 --:--:--     0
100 93.6M  100 93.6M    0     0  17.8M      0  0:00:05  0:00:05 --:--:-- 34.4M
Archive:  breast-cancer-cells-segmentation.zip
  inflating: Images/ytma10_010704_benign1_ccd.tif  
  inflating: Images/ytma10_010704_benign1_ccd.tif.xml  
  inflating: Images/ytma10_010704_benign2_ccd.tif  
  inflating: Images/ytma10_010704_benign2_ccd.tif.xml  
  inflating: Images/ytma10_010704_benign3_ccd.tif  
  inflating: Images/ytma10_010704_benign3_ccd.tif.xml  
  inflating: Images/ytma10_010704_malignant1_ccd.tif  
  inflating: Images/ytma10_010704_malignant1_ccd.tif.xml  
  inflating: Images/ytma10_010704_malignant2_ccd.tif  
  inflating: Images/ytma10_010704_malignant2_ccd.tif.xml  
  infl

## EDA

In [ ]:
!unzip breast-cancer-cells-segmentation.zip

Archive:  breast-cancer-cells-segmentation.zip
replace Images/ytma10_010704_benign1_ccd.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
imgs = glob(os.path.join("/content/Images", "*.tif"))

In [ ]:
print(os.getcwd())

In [ ]:
masks = [os.path.join("/content/Masks", i.rsplit("/",1)[-1].split("_ccd")[0]+".TIF") for i in imgs]

In [ ]:
print(masks)

In [ ]:
dataset_tuples = list(zip(imgs, masks))
random.shuffle(dataset_tuples)
train_tuples, test_tuples = train_test_split(dataset_tuples)

## Write Dataloaders and transform data

In [ ]:
SIZE = 512

In [ ]:
class BreastDataset(Dataset):

    def __init__(self, img_mask):
        self.img_mask = img_mask

    def __len__(self,):
        return len(self.img_mask)

    def __getitem__(self, idx):
        img_path, mask_path = self.img_mask[idx]
        image = self.get_transform(img_path)
        mask = self.get_transform(mask_path, normalize=False)
        mask[mask > 0] = 1
        return image,mask

    def transform_image(self, path, normalize=True,resize=(SIZE, SIZE)):
       image = io.imread(path)
       image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
       image = cv2.resize(image, resize)
       if normalize:
          return image/255
       return image

In [ ]:
train_dataset = BreastDataset(train_tuples)
test_dataset = BreastDataset(test_tuples)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## Get unet model

#### Create double convolution block

In [ ]:
class DoubleConv(nn.Module)
    def __init__(self, inch, ouch, kernel=3, padding=1):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(inch, ouch, kernel, padding=padding),
            nn.BatchNorm2d(ouch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ouch, ouch, kernel, padding=padding),
            nn.BatchNorm2d(ouch),
            nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.conv(x)
        return x

In [1]:
class Encoder(nn.Module):
  def __init__(self, inch, ouch):
    super().__init__()
    self.enconv = nn.Sequential(nn.MaxPool2d(2),
                                DoubleConv(inch, ouch))

    def forward(self,x):
      x = self.enconv(x)
      return x

SyntaxError: invalid syntax (<ipython-input-1-198ea377f0ee>, line 2)

In [3]:
class Decoder(nn.Module):
  def __init__(self, inch, ouch):
    super().__init__()
    self.upscaler = nn.ConvTranspose2d(inch, ouch//2, kernel_size=2, stride=2)
    self.conv = DoubleConv(inch, ouch)


    def forward(self, x1, x2):
      x1 = self.upscaler(x1)
      x1s = x1.size()
      x2s = x2.size()
      Xdiff = x2s[3] - x1s[3]
      Ydiff = x2s[2] - x1s[2]
      mdiffX = Xdiff - Xdiff//2
      mdiffY = Ydiff - Ydiff//2
      x1 = F.pad(x1, [Xdiff//2, mdiffX, Ydiff//2, mdiffY])
      x = torch.cat([x2, x1], dim=1)
      x = self.conv(x)
      return x

NameError: name 'nn' is not defined